In [7]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Finding XYZ

In [8]:
def find_xyz(ind_list, landmark):
  a = landmark[ind_list[0]]
  b = landmark[ind_list[1]]
  c = landmark[ind_list[2]]

  first = [a.x,a.y,a.z]
  mid = [b.x,b.y,b.z]
  end = [c.x,c.y,c.z]
  return first, mid, end

# Calculating Angle

In [9]:
def calculate_angle3D(a, b, c, direction):

  """
  calculate_angle3D is divided by left and right side because this function uses external product
  input : a,b,c -> landmarks with shape [x,y,z,visibility]
          direction -> int -1 or 1
                      -1 means Video(photo) for a person's left side and 1 means Video(photo) for a person's right side
  output : angle between vector ba and bc with range 0~360
  """

  # external product's z value
  external_z = (b[0] - a[0]) * (b[1] - c[1]) - (b[1] - a[1]) * (b[0] - c[0])

  a = np.array(a) # first
  b = np.array(b) # mid
  c = np.array(c) # end

  ba = b - a
  bc = b - c
  dot_result = np.dot(ba, bc)


  ba_size = np.linalg.norm(ba)
  bc_size = np.linalg.norm(bc)
  radi = np.arccos(dot_result / (ba_size * bc_size))
  angle = np.abs(radi * 180.0 / np.pi)

  # left side
  if external_z * direction > 0:
    angle = 360 - angle

  return angle

In [10]:
def calculate_angle2D(a, b, c, direction):

  """
  calculate_angle2D is divided by left and right side because this function uses external product
  input : a,b,c -> landmarks with shape [x,y,z,visibility]
          direction -> int -1 or 1
                      -1 means Video(photo) for a person's left side and 1 means Video(photo) for a person's right side
  output : angle between vector ba and bc with range 0~360
  """

  # external product's z value
  external_z = (b[0] - a[0]) * (b[1] - c[1]) - (b[1] - a[1]) * (b[0] - c[0])

  a = np.array(a[:2]) # first
  b = np.array(b[:2]) # mid
  c = np.array(c[:2]) # end

  ba = b - a
  bc = b - c
  dot_result = np.dot(ba, bc)


  ba_size = np.linalg.norm(ba)
  bc_size = np.linalg.norm(bc)
  radi = np.arccos(dot_result / (ba_size * bc_size))
  angle = np.abs(radi * 180.0 / np.pi)

  if external_z * direction > 0:
    angle = 360 - angle

  return angle

In [12]:
# Define function to calculate angle between landmarks
def calculate_angle(a, b, c):
    a = np.array(a) # landmark 26
    b = np.array(b) # landmark 15
    c = np.array(c) # landmark 25
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [11]:
def get_distance(lm_from, lm_to):
  x2 = (lm_from.x - lm_to.x)**2
  y2 = (lm_from.y-lm_to.y)**2
  return (x2+y2)**0.5

# Find Chest Pressure point

In [5]:

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        image_height, image_width, _ = image.shape

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Calculate the middle point between landmarks 11 and 12
            x1, y1 = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            x2, y2 = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            x_mid_1, y_mid_1 = (x1 + x2) / 2 * image_width, (y1 + y2) / 2 * image_height
            

            # Calculate the middle point between landmarks 23 and 24
            x1, y1 = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            x2, y2 = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            x_mid_2, y_mid_2 = (x1 + x2) / 2 * image_width, (y1 + y2) / 2 * image_height
            

            # Calculate the 2/3 point between the two middle points
            x, y = (2 * x_mid_1 + x_mid_2) / 3, (2 * y_mid_1 + y_mid_2) / 3

            # Fit the 1/3 point coordinate to the video size
            x, y = int(max(min(x, image_width), 0)), int(max(min(y, image_height), 0))

            # Draw a green circle on the 1/3 point
            cv2.circle(image, (x, y), 10, (0, 255, 0), -1)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('MediaPipe Pose', image)

        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


# Checking Condition for CPR Pressing distance

In [ ]:
# Define initial state and count as list
temp = ['up']
count = [0]

cap = cv2.VideoCapture(0)

# Define function to calculate angle between landmarks
def calculate_angle(a, b, c):
    a = np.array(a) # landmark 26
    b = np.array(b) # landmark 15
    c = np.array(c) # landmark 25
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return a

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        # Read a frame from the video
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        res=(int(width), int(height))

        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue
        
        # Flip the image horizontally
        image = cv2.flip(image, 1)
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks is not None:
            # Get the coordinates of landmark 26, 15, and 25
            landmark_left_knee = [results.pose_landmarks.landmark[26].x, results.pose_landmarks.landmark[26].y]
            landmark_left_wrist = [results.pose_landmarks.landmark[15].x, results.pose_landmarks.landmark[15].y]
            landmark_right_knee = [results.pose_landmarks.landmark[25].x, results.pose_landmarks.landmark[25].y]
            
            # Calculate the angle between landmark 26, 15, and 25
            angle = calculate_angle(landmark_left_knee, landmark_left_wrist, landmark_right_knee)
            
            # Check if the angle is less than 90 degrees
            if angle < 90:
                temp.append('up')
            
            # Check if the angle is greater than 105 degrees
            elif angle > 105:
                temp.append('down')
                
                # If the state of 'temp' changed from 'up' to 'down', increment 'count'
                if temp[-2] == 'up':
                    count.append(count[-1] + 1)
                
            # If the angle is between 90 and 105 degrees
            else:
                # Remove the 'Please pressure more' text if it is visible
                if len(temp) > 1 and temp[-2] == 'down':
                    cv2.putText(image, "", (int(image.shape[1]/2), int(image.shape[0]/2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                temp.append('up')
            
            # Display the 'Please pressure more' text if the state of 'temp' is 'up'
            if temp[-1] == 'up':
                cv2.putText(image, "Please pressure more", (int(image.shape[1]/2), int(image.shape[0]/2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # Display the count in the top left corner of the video
            cv2.putText(image, f"Count: {count[-1]}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        # Display the image
        cv2.imshow('MediaPipe Pose', image)
        
        # Check if the 'Esc' key was pressed
        if cv2.waitKey(5) & 0xFF == 27:
            break

# Release the VideoCapture object and close all windows
cap.release()
cv2.destroyAllWindows()

